In [1]:
from optimizer import *
from problem import *

def main():
    p, alg = readPlanAndCreate()  # Setup and create (problem, algorithm)
    conductExperiment(p, alg)     # Conduct experiment & produce results
    p.describe()                  # Describe the problem solved
    alg.displayNumExp()           # Total number of experiments
    alg.displaySetting()          # Show the algorithm settings
    p.report()                    # Report result

def readPlanAndCreate():
    parameters = readValidPlan()  # Read and store in 'parameters'
    p = createProblem(parameters)
    alg = createOptimizer(parameters)
    return p, alg

def readValidPlan():  # Gradient Descent cannot solve TSP
    while True:
        parameters = readPlan()
        if parameters['pType'] == 2 and parameters['aType'] == 4:
            print("Gradient Descent is only for numerical optimizations.")
        else:
            break
    return parameters

def readPlan():
    fileName = input("Enter the file name of experimental setting: ")
    infile = open(fileName, 'r')
    parameters = { 'pType':0, 'pFileName':'', 'aType':0, 'delta':0,
                   'limitStuck':0, 'alpha':0, 'dx':0, 'numRestart':0,
                   'limitEval':0, 'popSize':0, 'res':0, 'uXp':0, 
                   'mrF':0, 'XR':0, 'mR':0, 'numExp':0 }
    parNames = list(parameters.keys())
    for i in range(len(parNames)):
        line = lineAfterComments(infile)
        if parNames[i] == 'pFileName':
            parameters[parNames[i]] = line.rstrip().split(':')[-1][1:]
        else:
            parameters[parNames[i]] = eval(line.rstrip().split(':')[-1][1:])
    infile.close()
    return parameters             # Return a dictionary of parameters

def lineAfterComments(infile):    # Ignore lines beginning with '#'
    line = infile.readline()      # and then return the first line
    while line[0] == '#':         # with no '#'
        line = infile.readline()
    return line

def createProblem(parameters): ###
    # Create a problem instance (a class object) 'p' of the type as 
    # specified by 'pType', set the class variables, and return 'p'.
    pType = parameters['pType']
    if pType == 1:
        p = Numeric()
    else:
        p = TSP()
    p.setVariables(parameters)
    return p

def createOptimizer(parameters): ###
    # Create an optimizer instance (a class object) 'alg' of the type  
    # as specified by 'aType', set the class variables, and return 'alg'.
    optimizer ={ 1 : 'SteepestAscent()',
                 2 : 'FirstChoice()',
                 3 : 'Stochastic()',
                 4 : 'GradientDescent()',
                 5 : "SimulatedAnnealing()",
                 6 : "GA()"
                }
    aType = parameters['aType']
    alg = eval(optimizer[aType])
    alg.setVariables(parameters)
    return alg

def conductExperiment(p, alg):
    aType = alg.getAType()
    if 1 <= aType <= 4:
        alg.randomRestart(p)
    else:
        alg.run(p)
    bestSolution = p.getSolution()
    bestMinimum = p.getValue()    # First result is current best
    numEval = p.getNumEval()
    sumOfMinimum = bestMinimum    # Prepare for averaging
    sumOfNumEval = numEval        # Prepare for averaging
    sumOfWhen = 0                 # When the best solution is found
    if aType == 5: #5 <= aType <= 6?
        sumOfWhen = alg.getWhenBestFound()
    numExp = alg.getNumExp()
    for i in range(1, numExp):
        if 1 <= aType <= 4:
            alg.randomRestart(p)
        else:
            alg.run(p)
        newSolution = p.getSolution()
        newMinimum = p.getValue()  # New result
        numEval = p.getNumEval()
        sumOfMinimum += newMinimum
        sumOfNumEval += numEval
        if 5 <= aType < 6: #<= 6?
            sumOfWhen += alg.getWhenBestFound()
        if newMinimum < bestMinimum:
            bestSolution = newSolution  # Update the best-so-far
            bestMinimum = newMinimum
    avgMinimum = sumOfMinimum / numExp
    avgNumEval = round(sumOfNumEval / numExp)
    avgWhen = round(sumOfWhen / numExp)
    results = (bestSolution, bestMinimum, avgMinimum,avgNumEval, sumOfNumEval, avgWhen)
    p.storeExpResult(results)

main()

Enter the file name of experimental setting: a.txt

Number of cities: 30

City locations:
     (8, 31)    (54, 97)    (50, 50)    (65, 16)    (70, 47)
   (25, 100)    (55, 74)    (77, 87)     (6, 46)    (70, 78)
    (13, 38)   (100, 32)    (26, 35)    (55, 16)    (26, 77)
    (17, 67)    (40, 36)    (38, 27)     (33, 2)     (48, 9)
    (62, 20)    (17, 92)     (30, 2)    (80, 75)    (32, 36)
    (43, 79)    (57, 49)    (18, 24)    (96, 76)    (81, 39)

Number of experiments: 10

Search Algorithm: Genetic Algorithm

Population size: 100
Crossover rate: 0.1
Mutation rate: 0.9

Total number of evaluations: 1,027,620

Average objective value: 561.877
Average number of evaluations: 102,762

Best solution found:
    4   29   11   28    7    9   23    6    1   25
    5   21   15   14   12    8   10    0   27   17
   22   18   19   13    3   20   24   16    2   26

Best value: 535


In [2]:
main()

Enter the file name of experimental setting: exp.txt

Number of cities: 50

City locations:
      (1, 7)    (14, 92)    (45, 97)    (17, 60)    (22, 44)
     (4, 38)    (13, 73)    (79, 68)    (76, 95)    (62, 14)
    (25, 75)     (26, 9)    (88, 81)    (56, 65)    (64, 71)
    (92, 20)     (7, 20)     (8, 20)    (61, 39)    (17, 11)
    (10, 40)    (18, 72)    (89, 72)    (58, 25)    (57, 57)
    (66, 70)    (36, 72)    (89, 91)    (18, 90)    (72, 49)
    (82, 38)    (22, 26)    (36, 56)    (23, 44)    (45, 45)
     (7, 27)     (84, 6)    (32, 78)     (0, 29)    (64, 63)
    (45, 24)    (21, 81)    (37, 16)    (86, 57)    (65, 99)
    (25, 53)    (98, 24)    (83, 81)     (50, 5)    (58, 80)

Number of experiments: 10

Search Algorithm: Genetic Algorithm

Population size: 100
Crossover rate: 0.1
Mutation rate: 0.9

Total number of evaluations: 1,027,620

Average objective value: 1,148.392
Average number of evaluations: 102,762

Best solution found:
    0   11   38   16   35   17    5 

In [3]:
main()

Enter the file name of experimental setting: exp.txt

Number of cities: 100

City locations:
     (49, 3)    (74, 73)    (65, 36)    (39, 41)    (61, 99)
    (69, 44)    (88, 92)    (97, 28)    (53, 64)    (30, 77)
    (96, 62)    (61, 45)     (30, 3)    (66, 41)     (18, 9)
    (61, 64)    (28, 88)     (2, 72)    (80, 66)    (56, 38)
    (51, 16)     (18, 2)    (89, 18)    (67, 66)     (72, 6)
    (53, 32)    (29, 25)    (77, 69)    (89, 56)    (68, 88)
    (98, 53)    (36, 25)     (16, 0)    (20, 32)   (100, 10)
    (49, 49)    (85, 38)    (42, 52)     (3, 85)    (62, 77)
    (97, 87)    (75, 54)    (40, 19)    (32, 33)     (59, 1)
    (90, 43)    (62, 11)    (77, 14)    (88, 66)    (39, 32)
    (34, 69)    (12, 73)    (58, 88)    (34, 19)    (32, 45)
    (36, 36)    (84, 47)    (28, 18)    (23, 57)    (14, 52)
    (29, 38)     (0, 17)    (87, 96)    (61, 11)    (45, 56)
     (2, 60)    (97, 67)    (73, 70)    (49, 94)    (88, 55)
    (40, 55)    (23, 27)    (33, 68)    (70, 84)     

In [4]:
main()

Enter the file name of experimental setting: exp.txt

Objective function:
(x1 - 2) ** 2 +5 * (x2 - 5) ** 2 + 8 * (x3 + 8) ** 2 + 3 * (x4 + 1) ** 2 + 6 * (x5 - 7) ** 2

Search space:
 x1: (-30.0, 30.0)
 x2: (-30.0, 30.0)
 x3: (-30.0, 30.0)
 x4: (-30.0, 30.0)
 x5: (-30.0, 30.0)

Number of experiments: 10

Search Algorithm: Genetic Algorithm

Population size: 100
Number of bits for binary encoding: 10
Swap probability for uniform crossover: 0.2
Multiplication factor to 1/L for bit-flip mutation: 1

Total number of evaluations: 1,027,620

Average objective value: 4.390
Average number of evaluations: 102,762

Best solution found:
(1.992, 4.98, -7.969, -0.938, 7.031)
Best value: 0.027


In [5]:
main()

Enter the file name of experimental setting: exp.txt

Objective function:
1 + (x1 ** 2 + x2 ** 2 + x3 ** 2+ x4 ** 2 + x5 ** 2) / 4000 - math.cos(x1) * math.cos(x2 / math.sqrt(2)) * math.cos(x3 / math.sqrt(3)) * math.cos(x4 / math.sqrt(4)) * math.cos(x5 / math.sqrt(5))

Search space:
 x1: (-30.0, 30.0)
 x2: (-30.0, 30.0)
 x3: (-30.0, 30.0)
 x4: (-30.0, 30.0)
 x5: (-30.0, 30.0)

Number of experiments: 10

Search Algorithm: Genetic Algorithm

Population size: 100
Number of bits for binary encoding: 10
Swap probability for uniform crossover: 0.2
Multiplication factor to 1/L for bit-flip mutation: 1

Total number of evaluations: 1,027,620

Average objective value: 0.075
Average number of evaluations: 102,762

Best solution found:
(3.164, -4.512, -0.059, -0.41, 0.059)
Best value: 0.031


In [11]:
main()

Enter the file name of experimental setting: exp.txt

Objective function:
20 + math.e - 20 * math.exp(-(1/5) * math.sqrt((1/5) * (x1 ** 2 + x2 ** 2 + x3 ** 2 + x4 ** 2 + x5 ** 2))) - math.exp((1/5) * (math.cos(2 * math.pi * x1) + math.cos(2 * math.pi * x2) +math.cos(2 * math.pi * x3) +math.cos(2 * math.pi * x4) +math.cos(2 * math.pi * x5)))

Search space:
 x1: (-30.0, 30.0)
 x2: (-30.0, 30.0)
 x3: (-30.0, 30.0)
 x4: (-30.0, 30.0)
 x5: (-30.0, 30.0)

Number of experiments: 10

Search Algorithm: Genetic Algorithm

Population size: 100
Number of bits for binary encoding: 10
Swap probability for uniform crossover: 0.2
Multiplication factor to 1/L for bit-flip mutation: 1

Total number of evaluations: 1,027,620

Average objective value: 0.206
Average number of evaluations: 102,762

Best solution found:
(0.0, 0.0, 0.0, 0.0, 0.0)
Best value: -0.000
